In [1]:
import json
import random
import json
from PIL import Image
import os
import numpy as np
import cv2
from tqdm import tqdm
import pickle

from PIL import Image

In [2]:
def convert_prompt(mask_path,caption):
    
    caption=caption.replace('.','').strip()
    prompt_add="Please refer to this description:{}.Modify or correct it.".format(caption)

    return prompt_add

In [5]:
def get_single_data(prompt,img1,img2,answer):

    block={
            "conversations":[
                {
                    "from": "human",
                    "value": prompt
                },
                {
                    "from": "gpt",
                    "value": answer
                }
            ],
            "images": [
                img1,
                img2
            ]
        }
    
    return block

def get_multi_data(prompt,img1,img2,answer):

    block={
            "conversations":[
                {
                    "from": "human",
                    "value": prompt['Is']
                },
                {
                    "from": "gpt",
                    "value": answer['Is']
                },
                {
                    "from": "human",
                    "value": prompt['road']
                },
                {
                    "from": "gpt",
                    "value": answer['road']
                },
                {
                    "from": "human",
                    "value": prompt['build']
                },
                {
                    "from": "gpt",
                    "value": answer['build']
                },
                {
                    "from": "human",
                    "value": prompt['caption']
                },
                {
                    "from": "gpt",
                    "value": answer['caption']
                }
            ],
            "images": [
                img1,
                img2
            ]
        }
    
    return block

def getChageMask(mask_path):
    changeMask = Image.open(mask_path)
    changeMask = np.array(changeMask)
    return changeMask,changeMask.shape

def analyze_segmentation_mask(mask,flag=True):
    # 目标颜色定义
    if flag:
        ROAD = [128, 128, 128]
        BUILDING = [255, 255, 255]
        BACKGROUND = [0, 0, 0]
    else:
        ROAD = [255, 255,0]
        BUILDING = [255, 0, 0]
        BACKGROUND = [0, 0, 0]



import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

import pickle 
import json

extra_caption=pickle.load(open('/data/coding/datasets/extra/extra_info/extra_caption.pickle','rb'))
data=json.load(open('datasets/LEVIR-CC-dataset/LevirCCcaptions.json','r'))

from tqdm import tqdm
import numpy as np
outlist=[]

answer_dict={}

for i in tqdm(data['images']):

    filepath=i['filepath']

    filename=i['filename']
    answer=i['sentences'][0]['raw']

    truth_list=[]
    truth_list_all=[]
    

    for j in i['sentences']:
        truth_list.append(j['raw'].replace('.','').strip().split())

    out_compute=extra_caption[filename]
        

    smooth_fn = SmoothingFunction().method1 

    bleu_list=[]

    for j in truth_list:
        bleu_list.append(sentence_bleu([j], out_compute.split(), smoothing_function=smooth_fn,weights=(1,0,0,0))+sentence_bleu([j], out_compute.split(), smoothing_function=smooth_fn,weights=(0,1,0,0))+sentence_bleu([j], out_compute.split(), smoothing_function=smooth_fn,weights=(0,0,1,0))+sentence_bleu([j], out_compute.split(), smoothing_function=smooth_fn,weights=(0,0,0,1)))

    argmax=np.argmax(np.array(bleu_list))

    answer_dict[filename]=i['sentences'][argmax]['raw'].replace('.','').strip()

import pickle

data=json.load(open('datasets/LEVIR-CC-dataset/LevirCCcaptions.json','r'))

for i in tqdm(data['images']):

    filepath=i['filepath']
    filename=i['filename']

    mask_path='/data/coding/datasets/LEVIR-MCI-dataset/images/'+filepath+'/label/'+filename
    mask,img_shape=getChageMask(mask_path)
    has_target, target_counts, contours, road_contours_simplified, building_contours_simplified=analyze_segmentation_mask(mask)

    road_loc=determine_position(img_shape,road_contours_simplified)
    building_loc=determine_position(img_shape,building_contours_simplified)

    a_data=(has_target, target_counts, contours, road_contours_simplified, building_contours_simplified,img_shape,road_loc,building_loc)

    with open('/data/coding/datasets/LEVIR-CC-dataset/extra_info/'+filename,'wb') as f:
        pickle.dump(a_data,f)

100%|██████████| 10077/10077 [00:20<00:00, 480.55it/s]


In [6]:
from tqdm import tqdm
import numpy as np
outlist=[]

answer_dict={}

for i in tqdm(data['images']):

    filepath=i['filepath']



    filename=i['filename']
    answer=i['sentences'][0]['raw']

    truth_list=[]
    truth_list_all=[]
    

    for j in i['sentences']:
        truth_list.append(j['raw'].replace('.','').strip().split())

    out_compute=extra_caption[filename]
        

    smooth_fn = SmoothingFunction().method1  # 使用平滑方法

    bleu_list=[]

    for j in truth_list:
        bleu_list.append(sentence_bleu([j], out_compute.split(), smoothing_function=smooth_fn,weights=(1,0,0,0))+sentence_bleu([j], out_compute.split(), smoothing_function=smooth_fn,weights=(0,1,0,0))+sentence_bleu([j], out_compute.split(), smoothing_function=smooth_fn,weights=(0,0,1,0))+sentence_bleu([j], out_compute.split(), smoothing_function=smooth_fn,weights=(0,0,0,1)))

    #print(bleu_list)
    argmax=np.argmax(np.array(bleu_list))

    answer_dict[filename]=i['sentences'][argmax]['raw'].replace('.','').strip()

100%|██████████| 10077/10077 [00:20<00:00, 482.09it/s]


In [ ]:
import random

funtine_data_train_caption=[]           ### change caption
funtine_data_train_other_loc_road=[] 
funtine_data_train_other_loc_build=[] 
funtine_data_train_mutil=[]             ### 多轮对话

for i in tqdm(data['images']):

    ####
    multi_prompt={}
    multi_answer={}
    ####

    filepath=i['filepath']
    filename=i['filename']

    if filepath=='test':
        continue

    caption=extra_caption[filename]
    ppp=convert_prompt('/data/coding/datasets/extra/'+filename,caption)

    img1='/data/coding/datasets/LEVIR-CC-dataset/images/'+filepath+'/A/'+filename
    img2='/data/coding/datasets/LEVIR-CC-dataset/images/'+filepath+'/B/'+filename

    ### change caption

    caption_list=[]

    if i['changeflag']==0:
        answer='the scene is the same as before'

        block=get_single_data(ppp+"Please describe the changes in <image> and <image>",img1,img2,answer)

        funtine_data_train_caption.append(block)

    else:
        
        answer=answer_dict[filename]

        block=get_single_data(ppp+"Please describe the changes in <image> and <image>",img1,img2,answer) 
        
        funtine_data_train_caption.append(block)

      
    ######判断是否有改变

    if i['changeflag']==1:
        answer='Yes'
    else:
        answer='No'

    block=get_single_data("Please check if there have been any changes to the roads or buildings in these two images:<image> and <image>. Please answer yes or no.",img1,img2,answer)

    funtine_data_train_class.append(block)
    
    #if i['changeflag']==1:
    multi_prompt['Is']="Have there been any changes to the roads and buildings in these two images:<image> and <image>?"
    multi_answer['Is']=answer


    
    has_target, target_counts, contours, road_contours_simplified, building_contours_simplified,img_shape,road_loc,building_loc=pickle.load(open('/data/coding/datasets/LEVIR-MCI-dataset/extra_info/'+filename,'rb'))

    if target_counts['road']!=0:
        load_answer="There are a total of {} new road here.".format(target_counts['road'])
    else:
        load_answer='There are no changes in roads,'

    if target_counts['building']!=0:
        building_answer="There are a total of {} new buildings here.".format(target_counts['building'])
    else:
        building_answer='There are no changes in building.'

    answer=i['sentences'][0]['raw'].replace('.','').strip()
    if i['changeflag']==0:
        answer='There are no changes in roads or buildings between the two images'
    else:
        answer=load_answer+building_answer

    block=get_single_data("Please determine how many roads and buildings have changed between the two images:<image> and <image>",img1,img2,answer)

    funtine_data_train_count.append(block)


    if len(road_loc)!=0:
        counter = Counter(road_loc)
        load_answer="There are a total of {} new road here.There is".format(len(road_loc))
        for j in list(counter.keys()):
            load_answer+=" {} new road located in the {},".format(counter[j],j)
    else:
        load_answer='There are no changes in roads,'

    if len(building_loc)!=0:
        counter = Counter(building_loc)
        #print(counter)
        building_answer="There are a total of {} new buildings here.There is".format(len(building_loc))
        for j in list(counter.keys()):
            building_answer+=" {} new building located in the {},".format(counter[j],j)
        building_answer=building_answer[:-1]+'.' 
    else:
        building_answer='There are no changes in building.'

    answer=i['sentences'][0]['raw'].replace('.','').strip()

    if i['changeflag']==0:
        answer='There are no changes in roads or buildings between the two images'
    else:
        answer=load_answer+building_answer

    block=get_single_data("Please identify the location of roads and buildings in the changes in these two images:<image> and <image>",img1,img2,answer)

    funtine_data_train_loc.append(block)

    #break

    #####  road
    if len(road_loc)!=0:
        counter = Counter(road_loc)

        block_list=[]

        funtine_data_train_other_count_road.append(get_single_data("How many roads have been changed in these two images:<image> and <image>",img1,img2,str(len(road_loc))))

        block_list.append(["How many roads have been changed in these two images?",str(len(road_loc))])

        answer=''
        for j in list(counter.keys()):
            answer+=j+','

        answer=answer[:-1]
        
        funtine_data_train_other_loc_road.append(get_single_data("Please tell me where the roads have changed in these two images:<image> and <image>",img1,img2,answer))

        block_list.append(["Please tell me where the roads have changed in these two images",answer])

        bbb=random.choice(block_list)

        multi_prompt['road']=bbb[0]
        multi_answer['road']=bbb[1]

        
    else:
        answer='0'

        block_list=[]

        funtine_data_train_other_count_road.append(get_single_data("How many roads have been changed in these two images:<image> and <image>",img1,img2,answer))

        block_list.append(["How many roads have been changed in these two images?",answer])

        funtine_data_train_other_loc_road.append(get_single_data("Please tell me where the roads have changed in these two images:<image> and <image>,if no change please output: No change.",img1,img2,"No change"))

        block_list.append(["Please tell me where the roads have changed in these two images","No change"])
    
        bbb=random.choice(block_list)

        multi_prompt['road']=bbb[0]
        multi_answer['road']=bbb[1]
    
        ###

    #####  building
    if len(building_loc)!=0:

        block_list=[]

        counter = Counter(building_loc)
        funtine_data_train_other_count_build.append(get_single_data("How many buildings have been changed in these two images:<image> and <image>",img1,img2,str(len(building_loc))))
        
        block_list.append(["How many buildings have been changed in these two images?",str(len(building_loc))])
        
        answer=''
        for j in list(counter.keys()):
            answer+=j+','

        answer=answer[:-1]
        
        funtine_data_train_other_loc_build.append(get_single_data("Please tell me where the buildings have changed in these two images:<image> and <image>",img1,img2,answer))

        block_list.append(["Please tell me where the buildings have changed in these two images",answer])

        bbb=random.choice(block_list)

        multi_prompt['build']=bbb[0]
        multi_answer['build']=bbb[1]

    else:
        answer='0'
        block_list=[]
        funtine_data_train_other_count_build.append(get_single_data("How many buildings have been changed in these two images:<image> and <image>",img1,img2,answer))
        block_list.append(["How many roads have been changed in these two images?",answer])
        funtine_data_train_other_loc_build.append(get_single_data("Please tell me where the buildings have changed in these two images:<image> and <image>,if no change please output: No change.",img1,img2,"No change"))

        block_list.append(["Please tell me where the roads have changed in these two images","No change"])
    
        bbb=random.choice(block_list)

        multi_prompt['build']=bbb[0]
        multi_answer['build']=bbb[1]

        
  
    
        
    multi_prompt['caption']=ppp+'Please describe the changes.'
    multi_answer['caption']=answer_dict[filename]

    funtine_data_train_mutil.append(get_multi_data(multi_prompt,img1,img2,multi_answer))
    
    

  0%|          | 0/10077 [00:00<?, ?it/s]

100%|██████████| 10077/10077 [00:01<00:00, 9248.15it/s]


In [27]:
with open('muti_task_data/train_task_data/caption.json', 'w', encoding='utf-8') as json_file:
    json.dump(funtine_data_train_caption, json_file, ensure_ascii=False, indent=4)

In [7]:
import random

funtine_data_train_caption=[]           ### change caption
funtine_data_train_other_loc_road=[] 
funtine_data_train_other_loc_build=[] 
funtine_data_train_mutil=[]             ### 多轮对话

for i in tqdm(data['images']):

    ####
    multi_prompt={}
    multi_answer={}
    ####

    filepath=i['filepath']
    filename=i['filename']

    if filepath=='test':
        continue

    caption=extra_caption[filename]
    ppp=convert_prompt('/data/coding/datasets/extra/'+filename,caption)

    img1='/data/coding/datasets/LEVIR-CC-dataset/images/'+filepath+'/A/'+filename
    img2='/data/coding/datasets/LEVIR-CC-dataset/images/'+filepath+'/B/'+filename

    ### change caption

    caption_list=[]

    if i['changeflag']==0:
        answer='the scene is the same as before'

        block=get_single_data("Please describe the changes in <image> and <image>",img1,img2,answer)

        funtine_data_train_caption.append(block)

        caption_list.append(answer)

    else:
        
        answer=''
        
        for idx,j in enumerate(i['sentences']):
            answer=j['raw'].replace('.','').strip()

            block=get_single_data("Please describe the changes in <image> and <image>",img1,img2,answer) 
        
            funtine_data_train_caption.append(block)

            caption_list.append(answer)
    if len(caption_list) != 0:
        for kkk in caption_list:
            multi_prompt['caption']='Please describe the changes.'
            multi_answer['caption']=kkk

            funtine_data_train_mutil.append(get_multi_data(multi_prompt,img1,img2,multi_answer))
    
    

100%|██████████| 10077/10077 [00:02<00:00, 4431.44it/s]


In [12]:
with open('muti_task_data/test_task_data/loc_road.json', 'w', encoding='utf-8') as json_file:
    json.dump(funtine_data_train_other_loc_road, json_file, ensure_ascii=False, indent=4)
with open('muti_task_data/test_task_data/loc_build.json', 'w', encoding='utf-8') as json_file:
    json.dump(funtine_data_train_other_loc_build, json_file, ensure_ascii=False, indent=4)
with open('muti_task_data/test_task_data/mul.json', 'w', encoding='utf-8') as json_file:
    json.dump(funtine_data_train_mutil, json_file, ensure_ascii=False, indent=4)